In [1]:
import pandas as pd
import numpy as np
import time
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
def expand_column(df, test, column):
    #print(column + " - " + str(len(df[column].unique())))
    for value in df[column].unique():
        if (str(value) != 'nan'):
            #print(column + " -  " + str(value))
            new_column = column + '_' + str(value)
            df.loc[df[column] != value, new_column] = 0
            df.loc[df[column] == value, new_column] = 1
            test.loc[test[column] != value, new_column] = 0
            test.loc[test[column] == value, new_column] = 1

def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [3]:
df = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

enum_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 
                'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 
                'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 
                'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 
                'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'Fireplaces', 
                'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 
                'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

for col in enum_columns:
    expand_column(df, test, col)

df.drop(enum_columns, axis=1, inplace=True)
test.drop(enum_columns, axis=1, inplace=True)

df.fillna(df.mean(), inplace=True)
test.fillna(df.mean(), inplace=True)

df.drop('Id', axis=1, inplace=True)
textIds = test.loc[:,'Id']
test.drop('Id', axis=1, inplace=True)

Y = df.loc[:,'SalePrice']

X = df.drop('SalePrice', axis=1) # , inplace=True
X = normalize(X) # .iloc[:,0:8]

In [4]:
print(X.shape)
batch_size = X.shape[0]
input_dim = X.shape[1]
print(input_dim)

(1460, 322)
322


In [ ]:
X

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_CWD,SaleType_ConLw,SaleType_Con,SaleType_Oth,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_Partial,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family
0,0.150685,0.033420,0.949275,0.883333,0.122500,0.125089,0.000000,0.064212,0.140098,0.119780,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.202055,0.038795,0.753623,0.433333,0.000000,0.173281,0.000000,0.121575,0.206547,0.212942,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.160959,0.046507,0.934783,0.866667,0.101250,0.086109,0.000000,0.185788,0.150573,0.134465,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.133562,0.038561,0.311594,0.333333,0.000000,0.038271,0.000000,0.231164,0.123732,0.143873,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.215753,0.060576,0.927536,0.833333,0.218750,0.116052,0.000000,0.209760,0.187398,0.186095,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.219178,0.059899,0.876812,0.750000,0.000000,0.129695,0.000000,0.027397,0.130278,0.106012,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.184932,0.041057,0.956522,0.916667,0.116250,0.242558,0.000000,0.135702,0.275941,0.312070,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.167979,0.042450,0.731884,0.383333,0.150000,0.152197,0.021710,0.092466,0.181178,0.177375,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.102740,0.022529,0.427536,0.000000,0.000000,0.000000,0.000000,0.407534,0.155810,0.157871,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.099315,0.028605,0.485507,0.000000,0.000000,0.150780,0.000000,0.059932,0.162193,0.170491,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import tensorflow as tf

class NBatchLogger(tf.keras.callbacks.Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_batch_end(self, batch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log = ''
            for (k, v) in self.metric_cache.items():
                val = v / self.display
                #if abs(val) > 1e-3:
                metrics_log += ' - %s: %.4f' % (k, val)
                #else:
                #    metrics_log += ' - %s: %.4e' % (k, val)
            print('step: {}/{} ... {}'.format(self.step, self.params['steps'], metrics_log))
            self.metric_cache.clear()

In [ ]:
# print(X.isnull().any())
out_batch = NBatchLogger(display=10000)

from keras.callbacks import CSVLogger

ts = int(time.time())
csv_logger = CSVLogger('log_' + str(ts) + '.csv', append=True, separator='\t')

# create model
model = Sequential()
model.add(Dense(input_dim, activation='relu', input_dim=input_dim))
model.add(Dense(input_dim, activation='relu')) 
model.add(Dense(input_dim, activation='relu')) 
model.add(Dense(int(input_dim / 2), activation='relu')) #
model.add(Dense(1)) 

#, activation='sigmoid'#  relu
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, epochs=300000, batch_size=batch_size, verbose=0, callbacks=[csv_logger]) # , callbacks=[out_batch]

# evaluate the model
scores = model.evaluate(X, Y)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
test

In [ ]:
testX = normalize(test) # .iloc[:,0:8]
print(testX.shape)

In [ ]:
# Fit the model
testY = model.predict(testX, batch_size=100, verbose=1)
# evaluate the model
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

textIds['SalePrice'] = textY

textIds.to_csv(r'./data/result.csv', index=None, sep=',', mode='a')
